In [3]:
import sys
sys.path.insert(0, '../../') 
from project.data.datasets.ARG_KP_2021.processor import load_dataset
import pandas as pd

### load all arguments with quality score and ground truth kps 

In [44]:
#train_df = pd.read_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\train_complete.csv", index_col=None)
#dev_df =  pd.read_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\dev_complete.csv", index_col=None)
test_df =  pd.read_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\test_complete.csv", index_col=None)

In [45]:
print(len(test_df))
test_df.head(2)

3923


,idx,arg_id,argument,topic,stance,MACE-P,key_point_id,key_point,label,label_strict,label_relaxed,text,sentence1,sentence2
0,0,arg_0_0,Routine child vaccinations isn t mandatory since children don t spread the virus,Routine child vaccinations should be mandatory,-1,0.714195,kp_0_0,Routine child vaccinations or their side effects are dangerous,0.0,0.0,0.0,Routine child vaccinations isn t mandatory since children don t spread the virus Routine child vaccinations or their side effects are dangerous Routine child vaccinations should be mandatory,[CLS] Routine child vaccinations isn t mandatory since children don t spread the virus [SEP] Routine child vaccinations should be mandatory [SEP],[CLS] Routine child vaccinations or their side effects are dangerous [SEP] Routine child vaccinations should be mandatory [SEP]
1,1,arg_0_0,Routine child vaccinations isn t mandatory since children don t spread the virus,Routine child vaccinations should be mandatory,-1,0.714195,kp_0_1,Mandatory vaccination contradicts basic rights,0.0,0.0,0.0,Routine child vaccinations isn t mandatory since children don t spread the virus Mandatory vaccination contradicts basic rights Routine child vaccinations should be mandatory,[CLS] Routine child vaccinations isn t mandatory since children don t spread the virus [SEP] Routine child vaccinations should be mandatory [SEP],[CLS] Mandatory vaccination contradicts basic rights [SEP] Routine child vaccinations should be mandatory [SEP]


In [80]:
# import the potential kps, was prepare in a separate noteook (prepare_potential_kps.ipnyb)
all_pot_sent_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\pot_kps_cleaned_basic_filtering.csv', index_col=None)

In [46]:
# to be able to compare, we take only arguments with positive single label
test_ds_labeled = test_df[test_df['label'] == 1]
test_ds_labeled = test_ds_labeled.sample(frac=1).drop_duplicates(subset='arg_id')

### create cross args dataset

Get every possible pair to get matching score from best model. 

In [84]:
# all_pot_sent_df has potential kps sentences and test_ds_labeled has the arguments we want to generate kps for them

def get_paired_args(arguments_df, potential_kps, topic, stance):
    args_ids = []
    args = []
    kps = []
    q_scores = []
    
    filtered_arguments_df = arguments_df[arguments_df['topic'] == topic]
    filtered_potential_kps_df = potential_kps[potential_kps['topic'] == topic]
    
    filtered_arguments_df = filtered_arguments_df[filtered_arguments_df['stance'] == stance]
    filtered_potential_kps_df = filtered_potential_kps_df[filtered_potential_kps_df['stance'] == stance]
    
    unique_args_id = filtered_arguments_df['arg_id'].unique()
    
    for arg_id in unique_args_id:
        arg_row = filtered_arguments_df[filtered_arguments_df["arg_id"] == arg_id].iloc[0]
        for  index, s_row in filtered_potential_kps_df.iterrows():
            if arg_row["argument"] == s_row["sentence"]:
                continue
            args_ids.append(arg_id)
            args.append(arg_row["argument"])
            kps.append(s_row["sentence"])
            q_scores.append(s_row["q_score"])
    
    res_df = pd.DataFrame({
        "arg_id": args_ids,
        "argument": args,
        "key_point": kps,
        "kp_q_score": q_scores
    })
    
    res_df['topic'] = topic
    res_df['stance'] = stance
    return res_df

topics = test_ds_labeled['topic'].unique()
stances = [-1, 1]

cross_pairs_df = None
for topic in topics:
    for stance in stances:
        if cross_pairs_df is None:
            cross_pairs_df = get_paired_args(test_ds_labeled, all_pot_sent_df,  topic, stance)
        else:
            cross_pairs_df = pd.concat([cross_pairs_df, get_paired_args(test_ds_labeled, all_pot_sent_df, topic, stance)])

In [85]:
cross_pairs_df.head(2)

,arg_id,argument,key_point,kp_q_score,topic,stance
0,arg_1_49,Social media platforms should not be regulated by the government because this would restrict people from posting what comes to their minds,Freedom of speech should not be taken away.,0.793826,Social media platforms should be regulated by the government,-1
1,arg_1_49,Social media platforms should not be regulated by the government because this would restrict people from posting what comes to their minds,Government regulation of social media prevents people from expressing their freedom of speach.,0.906194,Social media platforms should be regulated by the government,-1


In [86]:
len(cross_pairs_df)

60695

In [87]:
cross_pairs_df.to_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_df_basic.csv', index=False)

### load best model 

Here we load best matcher model and score the pairs

In [88]:
import pandas as pd
test_cross_pairs_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_df_basic.csv', index_col=None)

In [89]:
len(test_cross_pairs_df)

60695

In [90]:
test_cross_pairs_df['idx'] = test_cross_pairs_df.index
test_cross_pairs_df.head(2)

,arg_id,argument,key_point,kp_q_score,topic,stance,idx
0,arg_1_49,Social media platforms should not be regulated by the government because this would restrict people from posting what comes to their minds,Freedom of speech should not be taken away.,0.793826,Social media platforms should be regulated by the government,-1,0
1,arg_1_49,Social media platforms should not be regulated by the government because this would restrict people from posting what comes to their minds,Government regulation of social media prevents people from expressing their freedom of speach.,0.906194,Social media platforms should be regulated by the government,-1,1


In [91]:
# process data to work with the classifier
from project.data.datasets.ARG_KP_2021.processor import preprocessing

test_cross_pairs_df['argument'] = test_cross_pairs_df['argument'].apply(lambda x: preprocessing(x))
test_cross_pairs_df['key_point'] = test_cross_pairs_df['key_point'].apply(lambda x: preprocessing(x))
test_cross_pairs_df['topic'] = test_cross_pairs_df['topic'].apply(lambda x: preprocessing(x))

test_cross_pairs_df['sentence1'] = '[CLS] ' + test_cross_pairs_df['argument'] + ' [SEP] ' + test_cross_pairs_df['topic'] + ' [SEP]'
test_cross_pairs_df['sentence2'] = '[CLS] ' + test_cross_pairs_df['key_point'] + ' [SEP] ' + test_cross_pairs_df['topic'] + ' [SEP]'

In [92]:
test_cross_pairs_df.to_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_clean_df_basic.csv', index=False)

In [93]:
# load best model
import torch
torch.cuda.empty_cache()
from project.models.baselines.classification_model import ClassificationModel
from project.models.common.nlp_configs import HFTransformerDataConfig
from transformers import AutoTokenizer
from project.data.nlp_data_module import NLPDataModule

test_data_path = r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_clean_df_basic.csv'
model_checkpoint_path = r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\models\good-models\epoch=1-step=807.ckpt"
model = ClassificationModel.load_from_checkpoint(checkpoint_path=model_checkpoint_path)
data_cfg = HFTransformerDataConfig(**{
    "batch_size": 32,
    "num_workers": 4,
    "train_file": test_data_path,
    "test_file": test_data_path
})
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model.model.config._name_or_path, use_fast=True)
datamodule = NLPDataModule(cfg=data_cfg, tokenizer=tokenizer)
datamodule.setup("test")
test_dataset = datamodule.ds["test"]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using custom data configuration default-6f3b44bea3547897


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to C:\Users\Bilal\.cache\huggingface\datasets\csv\default-6f3b44bea3547897\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?ba/s]

  0%|          | 0/61 [00:00<?, ?ba/s]

In [94]:
test_dataset.column_names

['arg_id',
 'argument',
 'attention_mask',
 'attention_mask_sent1',
 'attention_mask_sent2',
 'idx',
 'input_ids',
 'input_ids_sent1',
 'input_ids_sent2',
 'key_point',
 'kp_q_score',
 'sentence1',
 'sentence2',
 'stance',
 'topic']

In [96]:
# score match score for every possible pair
BATCH_SIZE = 32

test_ds_df = test_dataset.to_pandas()
test_ds_df.set_index('idx')
test_ds_df['score'] = -1

model.to('cuda')

def add_preds(examples):
    similarity, _, output = model._forward(examples, predict_with_gpu=True)
    for i, idx in enumerate(examples["idx"]):
        if output is not None:
            score = output[i][1].item()
        else:
            score = similarity[i].item()
        test_ds_df.iat[idx.item(), test_ds_df.columns.get_loc('score')] = score

test_dataset.map(lambda examples: add_preds(examples), batched=True, batch_size=BATCH_SIZE)



  0%|          | 0/1897 [00:00<?, ?ba/s]

Dataset({
    features: ['arg_id', 'argument', 'attention_mask', 'attention_mask_sent1', 'attention_mask_sent2', 'idx', 'input_ids', 'input_ids_sent1', 'input_ids_sent2', 'key_point', 'kp_q_score', 'sentence1', 'sentence2', 'stance', 'topic'],
    num_rows: 60695
})

In [97]:
test_ds_df.head(2)
test_ds_df.to_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_with_match_score_basic.csv', index=False)

In [1]:
import pandas as pd
test_ds_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_with_match_score_basic.csv', index_col=None)

In [2]:
print(len(test_ds_df))
test_ds_df.head(1)

60695


,arg_id,argument,attention_mask,attention_mask_sent1,attention_mask_sent2,idx,input_ids,input_ids_sent1,input_ids_sent2,key_point,kp_q_score,sentence1,sentence2,stance,topic,score
0,arg_1_49,Social media platforms should not be regulated...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,0,[ 0 10975 7454 104 742 3574 433 48...,[ 0 10975 7454 104 742 3574 433 48...,[ 0 10975 7454 104 742 7978 9 19...,Freedom of speech should not be taken away,0.793826,[CLS] Social media platforms should not be reg...,[CLS] Freedom of speech should not be taken aw...,-1,Social media platforms should be regulated by ...,0.64858


In [101]:
test_ds_df[['sentence1', 'sentence2', 'stance', 'topic', 'score']][test_ds_df['score']> 0.5].sort_values(by='score', ascending=True).head(1)

,sentence1,sentence2,stance,topic,score
7653,[CLS] social networks should be monitored mainly because of the interest that the owners may have in their publications and what consequences X types of publications bring to the population [SEP] Social media platforms should be regulated by the government [SEP],[CLS] Social media platforms should be regulated by the government to prevent bad minded criminals from having an effect on others [SEP] Social media platforms should be regulated by the government [SEP],1,Social media platforms should be regulated by the government,0.500003


In [3]:
# get top kps based on their number of matches (based on a threshold)
threshold = 0.5
result_df = test_ds_df[test_ds_df['score'] > threshold]
# choose best possible kp considering quality + match + rule-based
# first for every arg_id, get top 20 matches
# result_df = test_ds_df.groupby(by="arg_id").apply(lambda df: df.sample(frac=1, random_state=42).sort_values(by="score", ascending=False).head(1)).reset_index(drop=True)
# add combined average score quality+match/2 
# result_df['combined_score'] = (result_df['kp_q_score'] + result_df['score'] )/ 2
# second take the arg with top combined score from the 20 matches
# result_df = result_df.groupby(by="arg_id").apply(lambda df: df.sample(frac=1, random_state=42).sort_values(by="combined_score", ascending=False).head(1)).reset_index(drop=True)

In [4]:
len(result_df)

19135

In [5]:
print(len(result_df))
result_df.head(1)

19135


,arg_id,argument,attention_mask,attention_mask_sent1,attention_mask_sent2,idx,input_ids,input_ids_sent1,input_ids_sent2,key_point,kp_q_score,sentence1,sentence2,stance,topic,score
0,arg_1_49,Social media platforms should not be regulated...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,0,[ 0 10975 7454 104 742 3574 433 48...,[ 0 10975 7454 104 742 3574 433 48...,[ 0 10975 7454 104 742 7978 9 19...,Freedom of speech should not be taken away,0.793826,[CLS] Social media platforms should not be reg...,[CLS] Freedom of speech should not be taken aw...,-1,Social media platforms should be regulated by ...,0.64858


In [6]:
result_df.groupby(by=['topic', 'stance', 'key_point']).size()

topic                                           stance  key_point                                                                                                            
Routine child vaccinations should be mandatory  -1       There is a risk of a serious reaction or even death                                                                     48
                                                        A vaccine that has not been sufficiently tested and without knowledge of side effects is not recommended for children    35
                                                        As long as vaccines are not free of side effects  it cannot make them mandatory for our children                         37
                                                        Child vaccination is not necessary  and should not be mandatory  because vaccines are a lie                               1
                                                        Child vaccination isn t necessary and may result i

In [48]:
# get the top 10 most frequent KPs 

def get_topic_kps_counts(df):
    unique_topics = df['topic'].unique()
    unique_stances = [-1, 1]
    
    topics = []
    stances = []
    kps = []
    counts = []
    q_scores = []
    args = []
    for t in unique_topics:
        for stance in unique_stances:
            f_df = df[df['topic'] == t]
            f_df = f_df[f_df['stance'] == stance]   
            unique_kps = f_df['key_point'].unique()
           
            for kp in unique_kps:
                topics.append(t)
                stances.append(stance)
                kps.append(kp)
                #q_scores.append(f_df[f_df['key_point'] == kp].iloc[0]['kp_q_score'])
                counts.append(len(f_df[f_df['key_point'] == kp]))
                #args.append(list(f_df[f_df['key_point'] == kp]['arg_id']))

    ret_df = pd.DataFrame({
            "topic": topics,
            "stance": stances,
            "key_point": kps,
            #"q_score": q_scores,
            "counts": counts,
            #"args": args
        })
    return ret_df
    

In [20]:
counts_df = get_topic_kps_counts(result_df)

returning
506


In [27]:
len(counts_df)

506

In [28]:
# add word counts
counts_df['kp_word_count'] = counts_df['key_point'].str.split().str.len()

In [29]:
counts_df.head(1)

,topic,stance,key_point,q_score,counts,args,kp_word_count
0,Social media platforms should be regulated by ...,-1,Freedom of speech should not be taken away,0.793826,59,"[arg_1_49, arg_1_86, arg_1_13, arg_1_24, arg_1...",8


In [30]:
# sort them 
counts_df = counts_df.sort_values(by=["counts", "q_score", "kp_word_count"], ascending=(False, False, True)).reset_index(drop=True)

In [31]:
import difflib
s1=[1,8,3,9,4,9,3,8,1,2,3]
s2=[1,8,1,3,9,4,9,3,8,1,2,3]
sm=difflib.SequenceMatcher(None,s1,s2)
print(sm.ratio())    

0.9565217391304348


In [34]:
# get top kps, remove redundant by checking args overlap ratio


FILTER_OVERLAP_THRESHOLD = 0.8

top_filtered_sents = set()

for index, row in counts_df.iterrows():
    s1 = row['key_point']
    if len(top_filtered_sents) == 0:
        top_filtered_sents.add(s1)
        continue
    
    args_1 = list(row['args'])
    scores = []
    for s in top_filtered_sents:
        s_row = counts_df[counts_df['key_point'] == s].iloc[0]
        args_2 = list(s_row['args'])
        scores.append(difflib.SequenceMatcher(None,args_1,args_2).ratio())
    if max(scores) < FILTER_OVERLAP_THRESHOLD:
        top_filtered_sents.add(s1)
    

top_filtered_df = pd.DataFrame({"key_point": list(top_filtered_sents)})


In [35]:
top_filtered_df = pd.merge(top_filtered_df, counts_df, on='key_point')

In [38]:
len(counts_df)

314

In [39]:
counst_df = counts_df.groupby(by=["topic", "stance"]).apply(lambda df: df.sample(frac=1, random_state=42).sort_values(by=["counts", "q_score", "kp_word_count"], ascending=(False, False, True)).head(4)).reset_index(drop=True)

In [40]:
len(counst_df)

24

In [42]:
pd.set_option('display.max_colwidth', None)
counst_df[["topic", "stance", 'key_point', 'q_score', 'kp_word_count', 'counts']].head(24)

,topic,stance,key_point,q_score,kp_word_count,counts
0,Routine child vaccinations should be mandatory,-1,on the other hand there may be deadly side effects for children,0.832383,12,58
1,Routine child vaccinations should be mandatory,-1,Routine child vaccinations should not be necessary as children can t catch the disease,0.820775,14,45
2,Routine child vaccinations should be mandatory,-1,Child vaccination shouldn t be mandatory because the virus isn t effective in children,0.790943,14,42
3,Routine child vaccinations should be mandatory,-1,They should not be mandatory it is a personal decision,0.717764,10,38
4,Routine child vaccinations should be mandatory,1,I agree in order to be safer and keep them healthy,0.513677,11,123
5,Routine child vaccinations should be mandatory,1,Vaccinations are necessary to protect the health of the public as a whole and should be required by law,0.949995,19,92
6,Routine child vaccinations should be mandatory,1,Prevents a large number of diseases,0.657698,6,88
7,Routine child vaccinations should be mandatory,1,Should be mandatory because children are more vulnerable,0.636725,8,86
8,Social media platforms should be regulated by the government,-1,Social media platform should not beregulated by the government,0.728816,9,89
9,Social media platforms should be regulated by the government,-1,the fact that the government regulates social media platforms undermines freedom of expression and thought,0.968446,15,59


In [49]:
# comparing topics with labeled test dataset
test_ds_labeled_counts = get_topic_kps_counts(test_ds_labeled)

In [65]:
test_ds_labeled.columns

Index(['idx', 'arg_id', 'argument', 'topic', 'stance', 'MACE-P',
       'key_point_id', 'key_point', 'label', 'label_strict', 'label_relaxed',
       'text', 'sentence1', 'sentence2'],
      dtype='object')

In [50]:
# for every topic and stance
test_ds_labeled_counts = test_ds_labeled_counts.groupby(by=["topic", "stance"]).apply(lambda df: df.sample(frac=1, random_state=42).sort_values(by="counts", ascending=False).head(4)).reset_index(drop=True)

In [51]:
test_ds_labeled_counts.head(24)

,topic,stance,key_point,counts
0,Routine child vaccinations should be mandatory,-1,Routine child vaccinations or their side effects are dangerous,50
1,Routine child vaccinations should be mandatory,-1,The parents and not the state should decide,24
2,Routine child vaccinations should be mandatory,-1,Mandatory vaccination contradicts basic rights,21
3,Routine child vaccinations should be mandatory,-1,Routine child vaccinations are not necessary to keep children healthy,11
4,Routine child vaccinations should be mandatory,1,Children should not suffer from preventable diseases,44
5,Routine child vaccinations should be mandatory,1,Routine child vaccinations should be mandatory to prevent virus disease spreading,40
6,Routine child vaccinations should be mandatory,1,Routine child vaccinations are effective,23
7,Routine child vaccinations should be mandatory,1,Routine child vaccinations are necessary to protect others,20
8,Social media platforms should be regulated by the government,-1,Social media regulation harm freedom of speech and other democratic rights,50
9,Social media platforms should be regulated by the government,-1,Social media regulation harms privacy,14


# compare with the ground truth
 now we should compare result_df with test_ds_labeled , each having 500 arg_id

In [52]:
import rouge
# evaluate to ground truth using rouge
evaluator = rouge.Rouge(metrics=['rouge-l'],
                           limit_length=True,
                           length_limit_type='words',
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

In [55]:
# comparing test_ds_labeled_counts with counst_df
# for every topic, stance => compare predicted kp with every labeled kp and take max score
import numpy as np
from itertools import product
unique_topics = test_ds_labeled['topic'].unique()
unique_stances = [-1, 1]

topics = []
stances = []
scores = []

for t in unique_topics:
    for s in unique_stances:
        test_df = test_ds_labeled_counts[test_ds_labeled_counts['topic'] == t]
        test_df = test_df[test_df['stance'] == s]
        res_df = counst_df[counst_df['topic'] == t]
        res_df = res_df[res_df['stance'] == s]
        
        unique_predicted_kps = res_df['key_point'].unique()
        unique_labeled_kps = test_df['key_point'].unique()
        kp_scores = []
        for p_kp, l_kp in product(unique_predicted_kps, unique_labeled_kps) :
                score = evaluator.get_scores(p_kp, l_kp)
                kp_scores.append(score['rouge-l']['f'])    

        topics.append(t)
        stances.append(s)
        scores.append(np.mean(kp_scores))
        
    
results_df = pd.DataFrame({"topic": topics, "stance": stances,  "r_scores": scores})
    

In [56]:
results_df.head(6)

,topic,stance,r_scores
0,Routine child vaccinations should be mandatory,-1,0.203307
1,Routine child vaccinations should be mandatory,1,0.205086
2,The USA is a good country to live in,-1,0.084952
3,The USA is a good country to live in,1,0.210672
4,Social media platforms should be regulated by the government,-1,0.340336
5,Social media platforms should be regulated by the government,1,0.275814


In [99]:
# key points in labeled ds are completely new and not another arguments like the way we do it, difficult to get good rouge score with our method.
test_ds_labeled[test_ds_labeled['argument'] == test_ds_labeled[test_ds_labeled['arg_id'] == 'arg_0_99']['key_point'].iloc[0]]

,idx,arg_id,argument,topic,stance,MACE-P,key_point_id,label,key_point,text,sentence1,sentence2,subset
